# Telangana RERA Scraper

In [1]:
import os
import platform
import time

from selenium.webdriver import Chrome, Safari
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement

browser: WebDriver

if platform.system() == 'Darwin' and False:
	browser = Safari()
else:
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    browser = Chrome(executable_path="/usr/local/bin/chromedriver", options=chrome_options)

browser.implicitly_wait(20)
browser.get("http://rerait.telangana.gov.in/SearchList/Search")
page_size: WebElement = browser.find_element_by_id('PageSize')

# page_size.send_keys(Keys.BACKSPACE, Keys.BACKSPACE, "5", "0")
# time.sleep(10)

offset: int = 0
counter: int = 1 + offset

for i in range(offset):
    browser.find_element_by_xpath('//*[@id="btnNext"]').click()
print("Ready")

Ready


2. Scrape the 100 entries on the page

In [ ]:
import csv
import io
import shutil
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait

while counter <= 405:
	browser.switch_to.window(browser.window_handles[0])

	row_xpath = '//*[@id="gridview"]/div[1]/div/table/tbody/tr'

	output = io.StringIO()
	writer = csv.writer(output)
	writer.writerow([
		"Project Name",
		"Promoter Name",
		"Date Last Modified",
		"Net Area",
		"Total Built up Area",
		"Proposed Date of Completion"
		"Total Building Units",
		"Mandal",
		"Locality",
		"Pin Code",
		"Total No. of Apartments",
		"Total No. of Booked Apartments",
		"Total No. of Unbooked Apartments"
	])

	rows: list[WebElement] = browser.find_elements_by_xpath(row_xpath)
	for row_i, row in enumerate(rows):
		out_cells: list[WebElement] = row.find_elements_by_xpath('{}[{}]/td'.format(row_xpath, row_i + 1))
		# Intentionally using camelCase instead of snake_case as the schema uses camelCase
		srno = int(out_cells[0].get_attribute('innerHTML'))
		if srno == 3767 or srno == 207:
			continue
		print(srno)
		project_name = out_cells[1].get_attribute('innerHTML')
		promoter_name = out_cells[2].get_attribute('innerHTML')
		last_modified_date = out_cells[3].get_attribute('innerHTML')
		view_link: WebElement = out_cells[4].find_element_by_xpath('./b').find_element_by_xpath('./a')
		view_link.click()
		browser.switch_to.window(browser.window_handles[1])
		WebDriverWait(browser, 10).until(expected_conditions.presence_of_element_located((By.ID, "DivPartial")))
		try:
			mandal = browser.find_element_by_xpath('//*[@id="DivProject"]/div[4]/div[2]/div[2]/div/div[2]')\
				.get_attribute('innerHTML').strip()
			pincode = browser.find_element_by_xpath('//*[@id="DivProject"]/div[4]/div[2]/div[4]/div/div[2]')\
				.get_attribute('innerHTML').strip()
			locality = browser.find_element_by_xpath('//*[@id="DivProject"]/div[4]/div[2]/div[3]/div[1]/div[4]')\
				.get_attribute('innerHTML').strip()
			net_area = browser.find_element_by_xpath('//*[@id="DivProject"]/div[2]/div[2]/div[3]/div/div[2]')\
				.get_attribute('innerHTML').strip()
			total_building_units = browser.find_element_by_xpath('//*[@id="DivProject"]/div[2]/div[2]/div[3]/div/div[4]')\
				.get_attribute('innerHTML').strip()
			approved_built_up_area = browser.find_element_by_xpath('//*[@id="DivProject"]/div[3]/div[2]/div/div[2]')\
				.get_attribute('innerHTML').strip()
			proposed_date = browser.find_element_by_xpath('//*[@id="DivProject"]/div[1]/div[2]/div[4]/div/div[2]')\
				.get_attribute('innerHTML').strip()

			row_index = 3
			num_table: WebElement = browser.find_element_by_xpath('//*[@id="DivBuilding"]/div/table/tbody')
			rows: list[WebElement] = num_table.find_elements_by_xpath("./tr")
			total_apartments: int = 0
			total_booked_apartments: int = 0
			while row_index <= len(rows):
				row: WebElement = rows[row_index - 1]
				in_rows: list[WebElement] = row.find_elements_by_xpath("./td")[2]\
					.find_element_by_xpath("./table")\
					.find_element_by_xpath("./tbody")\
					.find_elements_by_xpath("./tr")
				for i in range(1, len(in_rows)):
					in_row = in_rows[i]
					total_apartments += int(in_row.find_elements_by_xpath("./td")[5].get_attribute('innerHTML'))
					total_booked_apartments += int(in_row.find_elements_by_xpath("./td")[6].get_attribute('innerHTML'))
				row_index += 4
			writer.writerow([
				project_name,
				promoter_name,
				last_modified_date,
				net_area,
				approved_built_up_area,
				proposed_date,
				total_building_units,
				mandal,
				locality,
				pincode,
				total_apartments,
				total_booked_apartments,
				(total_apartments - total_booked_apartments)
			])
		except:
			print("bruh")
		browser.close()
		browser.switch_to.window(browser.window_handles[0])

	browser.find_element_by_xpath('//*[@id="btnNext"]').click()
	os.makedirs('out', exist_ok=True)
	file = open("out/{}.csv".format(counter), mode="w")
	output.seek(0)
	shutil.copyfileobj(output, file, -1)
	file.close()
	counter += 1 # Use this in the file name

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
bruh
20
bruh
21
22
23
24
25
26
27
28
29
bruh
30
bruh
31
32
33
34
bruh
35
36
37
38
39
40
bruh
41
bruh
42
43
44
45
46
bruh
47
48
49
50
51
52
bruh
53
54
bruh
55
56
57
58
59
bruh
60
61
62
63
bruh
64
65
bruh
66
67
68
69
70
bruh
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
bruh
95
96
97
98
99
100
101
102
103
104
bruh
105
106
bruh
107
bruh
108
109
110
bruh
111
bruh
112
bruh
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
bruh
129
130
131
132
133
134
bruh
135
136
bruh
137
138
139
140
141
142
143
144
145
146
bruh
147
148
149
150
151
152
153
154
155
156
157
bruh
158
159
160
161
bruh
162
163
164
bruh
165
166
167
168
169
170
171
172
173
174
175
176
177
bruh
178
179
180
181
182
183
184
185
186
bruh
187
188
189
190
191
bruh
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
208
209
210
211
212
213
214
215
216
217
218
